In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
      .builder \
      .master("local[1]") \
      .appName("Desafio_4_Bulk")\
      .config("spark.logConf",'true')\
      .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.0') \
      .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1108)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1094)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:579)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1647)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:514)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:514)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:514)
	at org.apache.spark.api.java.JavaSparkC

In [ ]:
df = spark.read.json("data.json")

dfDados = df.select('current_observation.location','current_observation.latitude','current_observation.longitude',
'current_observation.observation_time','current_observation.weather','current_observation.temperature_string',
'current_observation.relative_humidity','current_observation.wind_string')

dfDados = dfDados.withColumnRenamed("location","Local de Observação") \
    .withColumnRenamed("observation_time",'horário de observação')\
    .withColumnRenamed("weather","tempo")\
    .withColumnRenamed("temperature_string","temperatura(°C & °F)")\
    .withColumnRenamed("relative_humidity","umidade")\
    .withColumnRenamed("wind_string","vento(Direção e força)")

dfDados.show()


NameError: name 'spark' is not defined

In [ ]:
dfDados.select.writeStream\
    .format('mongodb')\
    .option('spark.mongodb.connection.uri', 'mongodb://127.0.0.1:27017/')\
    .option('spark.mongodb.database', 'Lucas')\
    .option('spark.mongodb.collection', 'weather')\
    .option("checkpointLocation", checkpoint)\
    .outputMode("append").start().awaitTermination()